In [1]:
import numpy as np
import pandas as pd
from numba import njit, objmode, prange,deferred_type,cuda,jit,vectorize
from numba.typed import Dict
from numba.core import types
import time
import string
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt
import math
from BPnumba.DFFA import createDFFA,DFFA
from BPnumba.EDFFA import createEDFFA,EDFFA
from BPnumba.DABC import DABC, createDABC
from BPnumba.EDABC import createEDABC ,EDABC
from BPnumba.AGH import createAG,NAG,FlipMutation,GASearch
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.BPPH import create_Bin
from BPnumba.Individual import Ind, createR_intidivual,create_intidivual
from BPnumba.Poblation import CreateHeuristicPob,CreateRotHPob,EvalPob
from BPmodule.BPGen import createPA,GetInstance,createInstance,createPA
from BPnumba.BPPH import create_Bin

from collections import OrderedDict
from numba.experimental import jitclass

In [2]:
@njit
def factorial(n):
    return math.gamma(n+1)

In [3]:
for i in range(2,10,2):
    print(i)

2
4
6
8


In [4]:
factorial(50)

3.041409320171338e+64

In [5]:
@vectorize
def flip(x, y,pm,rot):
    if x > pm:
        return y
    else:
        if y == rot-1:
            return 0
        else: 
            return y+1

In [6]:
N = 50
pr=0.05
a = np.random.random(size=N)
b= np.random.randint(6,size=N)
print(a)
print(b)
flip(a,b,pr,6)

[0.82738608 0.8867776  0.13143091 0.45345953 0.43241336 0.17683647
 0.86870151 0.28817298 0.62667314 0.99502144 0.7517776  0.10701578
 0.23753305 0.04420331 0.66181079 0.354109   0.91641784 0.26824803
 0.74884291 0.43226484 0.34452357 0.75183956 0.3380186  0.58116387
 0.64077832 0.75140454 0.71388026 0.67425076 0.90886342 0.49347534
 0.27080982 0.16336588 0.02017487 0.61934294 0.94301792 0.08443567
 0.72135589 0.07652353 0.35451425 0.43009625 0.6353873  0.48829436
 0.63156088 0.15347767 0.7965493  0.6637613  0.53721306 0.94516181
 0.37915411 0.148293  ]
[3 3 1 0 4 0 4 1 1 5 1 4 2 1 5 3 0 3 5 1 1 2 2 0 4 1 3 3 2 5 0 4 3 3 0 0 4
 5 4 2 1 1 2 1 2 4 0 5 3 1]


array([3, 3, 1, 0, 4, 0, 4, 1, 1, 5, 1, 4, 2, 2, 5, 3, 0, 3, 5, 1, 1, 2,
       2, 0, 4, 1, 3, 3, 2, 5, 0, 4, 4, 3, 0, 0, 4, 5, 4, 2, 1, 1, 2, 1,
       2, 4, 0, 5, 3, 1], dtype=int64)

In [7]:
@cuda.jit
def add_array(a, b, c,pm):
    i = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    if i < a.size:
        if a[i]>pm:
            c[i] = a[i] - b[i]
        else:
            c[i] = 1
@njit
def FlipMutation(gen, pm:float,rotType:int):
    for i in np.arange(len(gen)):
        r = random.random()
        if r<=pm:
            if rotType == 6:
                aux = np.random.randint(rotType)
                if gen[i] == aux:
                    gen[i]+=1
                    if gen[i] >= rotType-1:
                        gen[i]=0
            else:
                if(gen[i]==0):
                    aux=1
                else:
                    aux=0
            gen[i] = aux
N = 50
pr=0.05
a = np.random.random(size=N)
b = a
print(a,b)
dev_c = cuda.device_array_like(a)

add_array[4, 8](a, b, dev_c,pr)
c = dev_c.copy_to_host()
print(c)

[0.06304468 0.85010965 0.89027626 0.4577143  0.85089979 0.41988353
 0.25331813 0.98443725 0.01668185 0.27065843 0.03226356 0.68393126
 0.3164178  0.504042   0.81807564 0.51309104 0.97010878 0.02413332
 0.69762183 0.72182999 0.45566888 0.44055259 0.81148299 0.94310762
 0.33638134 0.39873731 0.49021039 0.09812638 0.48499601 0.19535669
 0.84560813 0.49650223 0.02935615 0.6208775  0.69649646 0.24002339
 0.4115094  0.31791572 0.31492863 0.53234546 0.97629799 0.27307316
 0.17482221 0.2820342  0.30047044 0.17584113 0.56262004 0.4934088
 0.6233629  0.62137018] [0.06304468 0.85010965 0.89027626 0.4577143  0.85089979 0.41988353
 0.25331813 0.98443725 0.01668185 0.27065843 0.03226356 0.68393126
 0.3164178  0.504042   0.81807564 0.51309104 0.97010878 0.02413332
 0.69762183 0.72182999 0.45566888 0.44055259 0.81148299 0.94310762
 0.33638134 0.39873731 0.49021039 0.09812638 0.48499601 0.19535669
 0.84560813 0.49650223 0.02935615 0.6208775  0.69649646 0.24002339
 0.4115094  0.31791572 0.31492863 0.532

c:\Users\NicolasG\anaconda3\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (76) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


c:\Users\NicolasG\anaconda3\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [8]:
@cuda.jit
def FlipMutation(a, b, c,pm,rot):
    i = cuda.grid(1)
    if i < a.size:
        if a[i]>pm:
            c[i] = b[i]
        else:
            if c[i]==rot:
                c[i] = 0
            else:
                c[i] = b[i]+1

def mn(b):
    N = 50     
    a = np.random.random(size=N)
   
    print(b)
    dev_a = cuda.to_device(a)
    dev_b = cuda.to_device(b)

    threads_per_block = 256
    blocks_per_grid = (N + (threads_per_block - 1)) // threads_per_block
    dev_c = cuda.device_array_like(b)
    FlipMutation[blocks_per_grid, threads_per_block](dev_a, dev_b, dev_c,0.05,6)

    c = dev_c.copy_to_host()
    print(c)

In [ ]:
mn(np.random.randint(7,size=50))

In [ ]:
rn = np.random.randint(50,size=50,dtype=np.int64)
rn

In [ ]:
np.random.random(size=50)

In [ ]:
BR1 = GetInstance("BR1")
contenedor = BR1[0]
DataSet = BR1[1][0]

In [ ]:
from BPnumba.BPPH import DBLF
from BPnumba.Individual import CalcFi


@njit
def mn(cont, data,rot):
    UBOXES = InstaceBoxes(data)
    hpob = CreateRotHPob(pop =20, ITEMS = UBOXES.copy(), rotType=rot)
    EvalPob(hpob,UBOXES,cont,rot)
    ind = hpob[0]
    print(ind.fi)

In [ ]:
mn(contenedor, DataSet,6)

In [ ]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

numTest = 20
num = 5
agL = zerovec(num, 2*numTest)
dabcL = zerovec(num, 2*numTest)
ffaL = zerovec(num, 2*numTest)

pr_selec = 0.85
pr_cross = 0.75
maxItr=1000
pop = 50

In [ ]:

@njit(parallel=True)
def BenchAG(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    for i in prange(20):
        alg:NAG = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=0,mutType=heuristic)
        alg.Train(maxItr,  InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), NumbaList(boxes[i]), bin)
        lst[i] =alg.BestInd.fi

def TestAG(algorithms, problems, tp):
    nm:string=""
    if tp==1:
        nm="c1HAG"
        print("Empieza",nm)
    elif tp == 2:
        nm="c2HAG"
        print("Empieza",nm)
    else:
        nm="HAG"
        print("Empieza",nm)
    for problem in problems:
        for algoritm in algorithms:
            
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            BenchAG(datos.copy(),contenedor, maxItr, agL,tp)
            if problem!=1 and algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/AG/"+nm,agL)

In [ ]:
@njit(parallel=True)
def BenchABC(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    n=len(boxes[0])
    for i in prange(20):
        if(heuristic==0):
            alg:DABC = createDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi

        else:
            alg:EDABC = createEDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi
def TestEDABC(algorithms, problems,tp:int):
    nm:string=""
    if tp != 0:
        nm="c2EDABC"
        print("Inicia c2EDABC")
    else:
        nm="DABC"
        print("Inicia DABC")
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            BenchABC(datos.copy(),contenedor, maxItr, agL,heuristic =tp)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/ABC/"+nm,agL)
           

In [ ]:
@njit(parallel=True)
def BenchFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0
    ):
  
    for i in prange(20):
        alg:DFFA = createDFFA(mutType=0)
        alg.Train(Maxitr=maxItr,fireflyPob=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), datos= NumbaList(boxes[i]),contenedor= bin)
        lst[i] =alg.BestInd.fi

@njit(parallel=True)
def BenchEFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0,
    updateIndex:int=5,
    ):
   for i in prange(20): 
        alg:EDFFA = createEDFFA(mutType=0)
        alg.Train(updateIndex,maxItr, InstancePob(NumbaList( CreateHeuristicPob(pop,boxes[i],True)),boxes[i],bin),boxes[i],bin)
        lst[i] =alg.BestInd.fi

In [ ]:
def TestFFA(
    algorithms:list[int], 
    problems:list[int],
    tp:int, 
    evolutive:bool,
    updateIndex:int
    ):
    nm:string = ""
    if evolutive:
        if tp==1:
            nm="c1EDFFA"
            print("Inicia",nm)
        elif  tp==2:
            nm="c2EDFFA"
            print("Inicia", nm)
        else:
            nm="EDFFA"
            print("Inicia",nm)
    else:
        if tp==1:
            nm="c1DFFA"
            print("Inicia",nm)
        elif tp==2:
            nm="c2DFFA"
            print("Inicia",nm)
        elif tp==3:
            nm="invDFFA"
            print("Inicia",nm)
        else:
            nm="DFFA"
            print("Inicia",nm)
   
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            if not evolutive:
                BenchFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp)
            else:
                BenchEFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp,updateIndex=updateIndex)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("Termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/FFA/"+nm,agL)

In [ ]:
def Compile():
    UPDATE_INDEX:int=10
    ALGORITHM=[1]
    PROBLEM=[1]
    for _ in np.arange(4):
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM,
            tp=_,
            evolutive=True,
            updateIndex=UPDATE_INDEX,
        )
        if _ == 3: continue
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM, 
            tp=_,
            evolutive=False,
            updateIndex=UPDATE_INDEX,
        )
    TestEDABC(ALGORITHM,PROBLEM,0)
    TestEDABC(ALGORITHM,PROBLEM,1)

In [ ]:
Compile()

In [ ]:
TestEDABC([2],[3],0)

In [ ]:

TestEDABC([2],[3],1)


In [ ]:
UPDATE_INDEX:int=10


TestFFA(
        algorithms=[4],
        problems=[2],
        tp=0,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

TestFFA(
        algorithms=[4],
        problems=[2],
        tp=1,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )
TestFFA(
        algorithms=[4],
        problems=[2],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

In [ ]:

TestFFA(
        algorithms=[2],
        problems=[3],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
)

TestEDABC([2],[4],0)
TestEDABC([2],[4],1)

TestFFA(
    algorithms=[2],
    problems=[4],
    tp=0,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=1,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=2,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
        algorithms=[2],
        problems=[4],
        tp=0,
        evolutive=False,
        updateIndex=UPDATE_INDEX
)